In [2]:
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install opencv-python matplotlib

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-vc79v1_9
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-vc79v1_9
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36592 sha256=91aba53983a20396071c80da345e3b47889dc719f8f287b24502bbb903cada7b
  Stored in directory: /tmp/pip-ephem-wheel-cache-5nr146rj/wheels/10/cf/59/9ccb2f0a1bcc81d4fbd0e501680b5d088d690c6cfbc02dc99d
Successfully built segment_anything


In [3]:
import torch
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "/content/drive/MyDrive/sam_vit_h_4b8939.pth"
device = "cuda" if torch.cuda.is_available() else "cpu"
sam_model = sam_model_registry["vit_h"](checkpoint=sam_checkpoint)
sam_model.to(device)
sam_predictor = SamPredictor(sam_model)

print("SAM Model Loaded!")


/usr/local/lib/python3.10/dist-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


SAM Model Loaded!


In [4]:
SOURCE_VIDEO_PATH = "/content/17397133-trimmed.mp4"
TARGET_VIDEO_PATH = "/content/output_video.mp4"


In [5]:
import cv2
import time
import torch
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry

mask_generator = SamAutomaticMaskGenerator(sam_model)
cap = cv2.VideoCapture(SOURCE_VIDEO_PATH)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(TARGET_VIDEO_PATH, fourcc, frame_rate, (frame_width, frame_height))



In [15]:
import cv2
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
import numpy as np
import torch

frame_count = 0
total_object_count = 0
frame_skip = 5

tracked_objects = {}
line_y = frame_height - 100

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if frame_count % frame_skip != 0:
        frame_count += 1
        continue


    frame_resized = cv2.resize(frame, (frame_width // 2, frame_height // 2))
    frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)

    masks = mask_generator.generate(frame_rgb)
    object_count = 0
    for mask in masks:
        mask_image = mask["segmentation"].astype(np.uint8)
        if np.any(mask_image > 0):
            object_count += 1
            mask_image_resized = cv2.resize(mask_image, (frame_width, frame_height), interpolation=cv2.INTER_NEAREST)

            frame[mask_image_resized > 0] = (0, 255, 0)
    total_object_count += object_count
    cv2.putText(frame, f"Total Objects: {total_object_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
    cv2.line(frame, (0, line_y), (frame_width, line_y), (255, 0, 0), 2)
    out.write(frame)
    print(f"Processed frame {frame_count + 1}, Total Objects: {total_object_count}")
    frame_count += 1


cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Total objects counted across all frames: {total_object_count}")


/usr/local/lib/python3.10/dist-packages/segment_anything/build_sam.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)


Processed frame 1, Total Objects: 85
Processed frame 6, Total Objects: 171
Processed frame 11, Total Objects: 256
Processed frame 16, Total Objects: 346
Processed frame 21, Total Objects: 435
Processed frame 26, Total Objects: 525
Processed frame 31, Total Objects: 607
Processed frame 36, Total Objects: 694
Processed frame 41, Total Objects: 783
Processed frame 46, Total Objects: 864
Processed frame 51, Total Objects: 948
Processed frame 56, Total Objects: 1029
Processed frame 61, Total Objects: 1109
Processed frame 66, Total Objects: 1187
Processed frame 71, Total Objects: 1261
Processed frame 76, Total Objects: 1338
Processed frame 81, Total Objects: 1421
Processed frame 86, Total Objects: 1504
Processed frame 91, Total Objects: 1586
Processed frame 96, Total Objects: 1673
Processed frame 101, Total Objects: 1761
Processed frame 106, Total Objects: 1846
Processed frame 111, Total Objects: 1935
Processed frame 116, Total Objects: 2021
Processed frame 121, Total Objects: 2113
Processed